<a href="https://colab.research.google.com/github/sabnus307/My_first_repository/blob/master/split_sCO2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install CoolProp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 44.2 MB 2.0 MB/s 


In [ ]:
import CoolProp
print('CoolProp version:', CoolProp.__version__)
from CoolProp.CoolProp import PropsSI, get_global_param_string
import numpy as np
import matplotlib.pyplot as plt
import CoolProp.CoolProp as CP
import pandas as pd

#### bounsary consitions

#defining state values for HT loop
m_dot = 1 #assuming unit flow rate
x = 0.603 #split ratio
m_dot_HT = x  #kg/s # flowrate in HT loop
m_dot_LT = 1-x
TIT = 398+273.15 #K # turbine inlet temperature
m_eg = 69.8
T_EG_in = 538+273.15 #K #turbine exhaust gas inlet temperature
T_EG_mid = 289+273.15 #K #turbine exhaust gas middle temperature
T_EG_out = 73.8+273.15 #K #turbine exhaust gas outlet temperature
T_pinch_ext = 30+273.15 #K
T_pinch_int = 10+273.15 #K
P_H = 230*1e5 #bar to Pa #cycle max pressure
P_L = 57.3*1e5 #bar to Pa #cycle min pressure
T_con = 20+273.15 #K #condenser temperature
eta_pump = 0.80 #pump efficiency 
eta_turb = 0.85 #turbine efficiency
epsilon_R = 0.90 #recuperator effectiveness

print('High Temperature (HT) Loop Analysis')

#energy analysis
print('State point 1:')
P_1 = P_L
T_1 = T_con
h_1 = PropsSI( 'H','P',P_1,'T', T_1,'CO2')
s_1 = PropsSI('S','P',P_1,'T',T_1,'CO2')
print('h_1 =',h_1/1e3, 'kJ')
print('P_1 =',P_1/1e6, 'MPa')
print('T_1 =',T_1-273.15, 'C')
print('s_1 =',s_1/1e3, 'kJ/kg-K')
print('\n')

print('State point 2:')
P_2 = P_H
s_2s = s_1
h_2s = PropsSI( 'H','P',P_2,'S', s_2s,'CO2')
#eta_pump = (h_1-h_2s)/(h_1-h_2)
h_2 = h_1-((h_1-h_2s)/eta_pump)
T_2 = PropsSI( 'T','P',P_2,'H', h_2,'CO2')
s_2 = PropsSI( 'S','P',P_2,'H', h_2,'CO2')
print('h_2 =',h_2/1e3, 'kJ')
print('P_2 =',P_2/1e6, 'MPa')
print('T_2 =',T_2-273.15, 'C')
print('s_2 =',s_2/1e3, 'kJ/kg-K')
print('\n')

print('State point 4:')
P_4 = P_H
T_4 = TIT
h_4 = PropsSI( 'H','P',P_4,'T', T_4,'CO2')
s_4 = PropsSI('S','P',P_4,'T',T_4,'CO2')
print('h_4 =',h_4/1e3, 'kJ')
print('P_4 =',P_4/1e6, 'MPa')
print('T_4 =',T_4-273.15, 'C')
print('s_4 =',s_4/1e3, 'kJ/kg-K')
print('\n')

print('State point 5:')
P_5 = P_L
s_5s = s_4
h_5s = PropsSI( 'H','P',P_5,'S', s_5s,'CO2')
#eta_turb = (h_4-h_5)/(h_4-h_5s)
h_5 = h_4-(eta_turb*(h_4-h_5s))
T_5 = PropsSI( 'T','P',P_5,'H', h_5,'CO2')
s_5 = PropsSI('S','P',P_5,'T',T_5,'CO2')
print('h_5 =',h_5/1e3, 'kJ')
print('P_5 =',P_5/1e6, 'MPa')
print('T_5 =',T_5-273.15, 'C')
print('s_5 =',s_5/1e3, 'kJ/kg-K')
print('\n')

print('Rate of maximum heat exchange:')
P_6 = P_L
T_6a = T_2 #assuming T2 = T6
h_6a = PropsSI( 'H','P',P_6,'T', T_6a,'CO2')
Q_max = m_dot*(h_5-h_6a) #J-kg/s #rate of maximum heat exchange
print('Q_max:', Q_max/1e3,'kJ-kg/s')
print('\n')

print('State point 3:')
P_3 = P_H
#epsilon_R = (m_dot*(h_3-h_2))/Q_max
h_3 = h_2 + ((epsilon_R*Q_max)/m_dot_HT)
T_3 = PropsSI( 'T','P',P_3,'H', h_3,'CO2')
s_3 = PropsSI('S','P',P_3,'H',h_3,'CO2')
print('h_3 =',h_3/1e3, 'kJ')
print('P_3 =',P_3/1e6, 'MPa')
print('T_3 =',T_3-273.15, 'C')
print('s_3 =',s_3/1e3, 'kJ/kg-K')
print('\n')

print('State point 6:')
P_6 = P_L
#epsilon_R = (m_dot*(h_5-h_6))/Q_max
h_6 = h_5 -((epsilon_R*Q_max)/m_dot)
T_6 = PropsSI('T','P',P_6,'H', h_6,'CO2')
s_6 = PropsSI('S','P',P_6,'H',h_6,'CO2')
print('h_6 =',h_6/1e3, 'kJ')
print('P_5 =',P_6/1e6, 'MPa')
print('T_6 =',T_6-273.15, 'C')
print('s_6 =',s_6/1e3, 'kJ/kg-K')
print('\n')


print('exhaust gas calculation heater 1')
P_eg = P_L #assumption
h_eg_in = PropsSI('H','T',T_EG_in,'P',P_eg,'CO2') #enthalpy of gas at heater 1 inlet
print('h_eg_in =',h_eg_in/1e3, 'kJ')
h_eg_mid = PropsSI('H','T',T_EG_mid,'P',P_eg,'CO2') #enthalpy of gas at heater 1 outlet
print('h_eg_mid =',h_eg_mid/1e3, 'kJ')
h_diff_1 = h_eg_in - h_eg_mid #enthalpy difference in heater 1
print('h_diff_1 =',h_diff_1/1e3, 'kJ')
print('\n')

print('m_dot_CO2 Calculation')
Q_eg_1 = m_eg * h_diff_1 #heat exchange in heater 1
print('Q_eg_1 =',Q_eg_1/1e3, 'kJ')
#Heat Exchanged = m_dot * (h4- h3)
m_dot_actual = Q_eg_1/ (h_4 - h_3) # actual flowrate of CO2
print('m_dot_actual =',m_dot_actual, 'kg/s')

m_dot_HT_actual = m_dot_actual * m_dot_HT # actual flowrate of CO2 via recuperator
print('m_dot_actual =',m_dot_actual, 'kg/s')

m_dot_LT_actual = m_dot_actual * m_dot_LT # actual flowrate of CO2 via heater 2
print('m_dot_actual =',m_dot_actual, 'kg/s')
print('\n')

print('exhaust gas calculation heater 2')
P_eg = P_L
print('h_eg_mid =',h_eg_mid/1e3, 'kJ')
h_eg_out = PropsSI('H','T',T_EG_out,'P',P_eg,'CO2')
print('h_eg_out =',h_eg_out/1e3, 'kJ')
h_diff_2 = h_eg_mid - h_eg_out #enthalpy difference in heater 2 for ex gas
print('h_diff_2 =',h_diff_2/1e3, 'kJ')
Q_eg_2 = m_eg * h_diff_2  #heat exchange in heater 1
print('Q_eg_2 =',Q_eg_2/1e3, 'kJ')
print('\n')

print('Low temperature heater cycle calculation')
h_diff_3 = Q_eg_2 / m_dot_LT_actual #enthalpy difference in heater 2 for CO2
h_3L = h_2 + h_diff_3 
P_3L = P_H
T_3L = PropsSI('T','P',P_3L,'H', h_3L,'CO2')
s_3L = PropsSI('S','P',P_3L,'H',h_3L,'CO2')
print('h_3L =',h_3L/1e3, 'kJ')
print('P_3L =',P_3L/1e6, 'MPa')
print('T_3L =',T_3L-273.15, 'C')
print('s_3L =',s_3L/1e3, 'kJ/kg-K')
print('\n')



#work calculation
W_p = m_dot_actual*(h_2-h_1) 
print('Pump rate of work (input):',W_p/1e3,'kJ-kg/s')
W_t = m_dot_actual*(h_4-h_5)
print('Turbine rate of work (output):',W_t/1e3,'kJ-kg/s')
W_net = (W_t-W_p)
print('Net work (output):',W_net/1e3,'kJ-kg/s')
Q_h = Q_eg_1 + Q_eg_2
print('Heater rate of work (input):',Q_h/1e3,'kJ-kg/s')
Q_c= m_dot_actual*(h_6-h_1) 
print('Heater rate of work (output):',Q_c/1e3,'kJ-kg/s')
print('\n')
eta_cyc = (W_t-W_p)/Q_h
print('Cycle thermal efficiency:',eta_cyc)

Q_eg_max = 40.9e6 #W
h_diff_4 = Q_eg_max/m_eg #enthalpy difference

eta_hr = (h_eg_in- h_eg_out)/h_diff_4
print('Heat recovery efficiency:',eta_hr)

#eta_hr = (h_4-h_3)/h_diff
#print('Heat recovery efficiency:',eta_hr)

n_sys = eta_hr*eta_cyc
print('System thermal efficiency:',n_sys)

eta_cyc_ref = 0.29
eta_hr_ref = 0.896 
n_sys_ref = 0.26
W_net_ref = 10640e3

eta_cyc_error = abs(eta_cyc -eta_cyc_ref)
print('Cycle thermal efficiency error:',round(eta_cyc_error *100/eta_cyc_ref,2),'%')

eta_hr_error = abs(eta_hr -eta_hr_ref)
print('Heat Recovery efficiency error',round(eta_hr_error *100/eta_hr_ref,2),'%')

W_net_error = abs(W_net - W_net_ref)
print('Net work error:',round(W_net_error *100/W_net_ref,2),'%')

n_sys_error = abs(n_sys - n_sys_ref)
print('Sytem efficiency error:',round(n_sys_error *100/n_sys_ref,2),'%')

CoolProp version: 6.4.1
High Temperature (HT) Loop Analysis
State point 1:
h_1 = 255.86249820761472 kJ
P_1 = 5.73 MPa
T_1 = 20.0 C
s_1 = 1.1877062900124837 kJ/kg-K


State point 2:
h_2 = 282.12794013842154 kJ
P_2 = 23.0 MPa
T_2 = 43.749426219845134 C
s_2 = 1.204347085292089 kJ/kg-K


State point 4:
h_4 = 844.3716988128718 kJ
P_4 = 23.0 MPa
T_4 = 398.0 C
s_4 = 2.457762984016449 kJ/kg-K


State point 5:
h_5 = 715.3564763477007 kJ
P_5 = 5.73 MPa
T_5 = 261.511589439267 C
s_5 = 2.5011918023228246 kJ/kg-K


Rate of maximum heat exchange:
Q_max: 252.92948553552537 kJ-kg/s


State point 3:
h_3 = 659.6346349675639 kJ
P_3 = 23.0 MPa
T_3 = 251.8325094178898 C
s_3 = 2.1469405044603525 kJ/kg-K


State point 6:
h_6 = 487.71993936572784 kJ
P_5 = 5.73 MPa
T_6 = 60.92667129605667 C
s_6 = 1.9640393671721663 kJ/kg-K


exhaust gas calculation heater 1
h_eg_in = 1032.3943401305903 kJ
h_eg_mid = 745.7422246003807 kJ
h_diff_1 = 286.65211553020964 kJ


m_dot_CO2 Calculation
Q_eg_1 = 20008.317664008628 kJ
m_do